In [35]:
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import altair as alt

In [51]:
# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [81]:
def remove_redundant(input_text, output_text):
    if output_text.startswith(input_text):
        return output_text[len(input_text):].strip()
    return output_text

In [82]:
data = pd.read_json('base_input_output_pairs.json')
data.rename(columns={'output': 'output_base'}, inplace=True)
data['output_base'] = data.apply(lambda row: remove_redundant(row['input'], row['output_base']), axis=1)

In [83]:
data2 = pd.read_json('input_output_pairs.json')
data2.rename(columns={'output': 'output_fine_tuned'}, inplace=True)
data2['output_fine_tuned'] = data2.apply(lambda row: remove_redundant(row['input'], row['output_fine_tuned']), axis=1)

In [84]:
s_class = ["Zero-shot",
           "Zero-shot",
           "Zero-shot",
           "One-shot",
           "One-shot",
           "One-shot",
           "Few-shot",
           "Few-shot",
           "Few-shot",
           "Negative",
           "Negative",
           "Negative",
           "COT",
           "COT",
           "COT"]
s_class = pd.Series(s_class, name="prompt_class")

In [85]:
data3 = pd.read_json('input_output_pairs.json2')
data3.rename(columns={'output': 'output_fine_tuned_context'}, inplace=True)
data3['output_fine_tuned_context'] = data3.apply(lambda row: remove_redundant(row['input'], row['output_fine_tuned_context']), axis=1)

In [86]:
data_final = pd.merge(data, data2)
data_final = pd.merge(data_final, data3)

In [87]:
data_final['prompt_type'] = s_class

In [88]:
data_final.head()

input  \
0  List the most pressing topics regarding regula...   
1  Who owns material generated by a company’s lar...   
2  Describe how China and the United States are a...   
3  Task: You are a chat bot called AI PolicyChat....   
4  Task: You are a chat bot called AI PolicyChat....   

                                         output_base  \
0  The most pressing topics regarding regulation ...   
1  The question of who owns the intellectual prop...   
2  What are the main differences between the Chin...   
3  The government can ensure that people negative...   
4  AI regulators attempt to enforce their regulat...   

                                   output_fine_tuned  \
0  - Defining AI accurately\n- The role of AI in ...   
1  ### Question: What ethical principles guide th...   
2  ### Answer: China is moving quickly to draft n...   
3  The government can help people negatively affe...   
4  AI regulators enforce through compliance and i...   

                           output_fine_tuned_context prompt_type  
0  Describe the current regulatory landscape for ...   Zero-shot  
1  Question:  Who owns the intellectual property ...   Zero-shot  
2  ### Question: Why is regulating Generative AI ...   Zero-shot  
3  AI can exacerbate societal harms and threaten ...    One-shot  
4  AI regulators attempt to enforce their regulat...    One-shot

### Calculate Similarity

In [89]:
def similarity_score(row):
    embedding1 = model.encode([row['output_base']])
    embedding2 = model.encode([row['output_fine_tuned']])
    embedding3 = model.encode([row['output_fine_tuned_context']])
    #print("Embedding 1: ", embedding1)
    #print("Embedding 2: ", embedding2)
    similarity1 = cosine_similarity([embedding1[0]], [embedding2[0]])
    #print(similarity1)
    similarity2 = cosine_similarity([embedding1[0]], [embedding3[0]])
    similarity3 = cosine_similarity([embedding2[0]], [embedding3[0]])
    return [similarity1[0][0], similarity2[0][0], similarity3[0][0]]

In [90]:
data_final['similarity'] = data_final.apply(similarity_score, axis=1)
data_final[['similarity_base_fine_tuned', 'similarity_base_fine_tuned_context', 'similarity_fine_tuned_fine_tuned_context']] = pd.DataFrame(data_final['similarity'].tolist(), index=data_final.index)
data_final = data_final.reset_index()

In [91]:
data_final = data_final.drop(columns='similarity')
data_final.head()

index                                              input  \
0      0  List the most pressing topics regarding regula...   
1      1  Who owns material generated by a company’s lar...   
2      2  Describe how China and the United States are a...   
3      3  Task: You are a chat bot called AI PolicyChat....   
4      4  Task: You are a chat bot called AI PolicyChat....   

                                         output_base  \
0  The most pressing topics regarding regulation ...   
1  The question of who owns the intellectual prop...   
2  What are the main differences between the Chin...   
3  The government can ensure that people negative...   
4  AI regulators attempt to enforce their regulat...   

                                   output_fine_tuned  \
0  - Defining AI accurately\n- The role of AI in ...   
1  ### Question: What ethical principles guide th...   
2  ### Answer: China is moving quickly to draft n...   
3  The government can help people negatively affe...   
4  AI regulators enforce through compliance and i...   

                           output_fine_tuned_context prompt_type  \
0  Describe the current regulatory landscape for ...   Zero-shot   
1  Question:  Who owns the intellectual property ...   Zero-shot   
2  ### Question: Why is regulating Generative AI ...   Zero-shot   
3  AI can exacerbate societal harms and threaten ...    One-shot   
4  AI regulators attempt to enforce their regulat...    One-shot   

   similarity_base_fine_tuned  similarity_base_fine_tuned_context  \
0                    0.679636                            0.803659   
1                    0.355390                            0.819518   
2                    0.669402                            0.614450   
3                    0.772910                            0.595537   
4                    0.849945                            0.762934   

   similarity_fine_tuned_fine_tuned_context  
0                                  0.709741  
1                                  0.378594  
2                                  0.634551  
3                                  0.668075  
4                                  0.799199

In [92]:
data_final.describe()

index  similarity_base_fine_tuned  \
count  15.000000                   15.000000   
mean    7.000000                    0.585984   
std     4.472136                    0.209943   
min     0.000000                    0.043731   
25%     3.500000                    0.462042   
50%     7.000000                    0.669402   
75%    10.500000                    0.696268   
max    14.000000                    0.849945   

       similarity_base_fine_tuned_context  \
count                           15.000000   
mean                             0.585062   
std                              0.248872   
min                              0.004420   
25%                              0.554798   
50%                              0.614450   
75%                              0.764545   
max                              0.819518   

       similarity_fine_tuned_fine_tuned_context  
count                                 15.000000  
mean                                   0.633448  
std                                    0.124114  
min                                    0.378594  
25%                                    0.611432  
50%                                    0.668075  
75%                                    0.713964  
max                                    0.799199

In [98]:
alt.Chart(data_final).mark_bar().encode(
    y=alt.X("index:N").title("Prompt Number"),
    x=alt.Y("similarity_base_fine_tuned:Q").title("Cosine Similarity"),
    color="prompt_type"
).properties(
    title="Model Output Cosine Similarity (Base & Fine-Tuned Models) by Input Prompt"
).interactive()

alt.Chart(...)

In [99]:
alt.Chart(data_final).mark_bar().encode(
    y=alt.X("index:N").title("Prompt Number"),
    x=alt.Y("similarity_base_fine_tuned_context:Q").title("Cosine Similarity"),
    color="prompt_type"
).properties(
    title="Model Output Cosine Similarity (Base & Fine-Tuned w/ Context Models) by Input Prompt"
).interactive()

alt.Chart(...)

In [95]:
alt.Chart(data_final).mark_bar().encode(
    y=alt.X("index:N").title("Prompt Number"),
    x=alt.Y("similarity_fine_tuned_fine_tuned_context:Q").title("Cosine Similarity"),
    color="prompt_type"
).properties(
    title="Model Output Cosine Similarity (Fine-Tuned & Fine-Tuned w/ Context Models) by Input Prompt"
).interactive()

alt.Chart(...)

In [97]:
data_final.to_csv("prompt_engineering_new_prompts.csv")